# Step 3: Build your data tensor

Use this notebook to arrange your normalized data into tensor format using the [xarray](https://docs.xarray.dev/en/stable/) package. 

Please note that your input data should already have been normalized, and should be saved in a csv file using [tidy format](https://tidyr.tidyverse.org/articles/tidy-data.html). At a minimum your input csv should have five columns:
1. A column that corresponds to the first mode of your tensor. In metatranscriptomic data this column might indicate gene ID.
    - This first mode should generally be the longest in your tensor, and the one that corresponds to the variable you want clustered (e.g. genes in the case of metatranscriptomics data). The sparsity penalty (`lambda`) will be applied to this mode.
1. A column that corresponds to the second mode of your tensor. In metatranscriptomic data this column might indicate taxon ID.
1. A column that corresponds to the third mode of your tensor. This column should indicate sample ID.
    - **IMPORTANT: Sample IDs should be identical for different replicates of the same sample condition (see example below).**
1. A column that indicates the replicate ID of the sample.
1. A column that corresponds to the normalized data. If you normalized your data with sctransform (as laid out in jupyter notebook [2-normalize-sctransform.ipynb](https://github.com/blasks/barnacle-boilerplate/blob/main/2-normalize-sctransform.ipynb), this column will correspond to the residual.

Here's a snippet of how an example csv might be arranged:

| gene_id | taxon_id   | sample_id | replicate | residual |
|---------|------------|-----------|-----------|----------|
| K03839  | P. marinus | sample1   | A         | 3.02     |
| K03839  | P. marinus | sample1   | B         | 3.31     |
| K03839  | P. marinus | sample1   | C         | 3.18     |
| K03839  | P. marinus | sample2   | A         | -1.24    |
| ...     | ...        | ...       | ...       | ...      |
| K03320  | S. marinus | sample9   | C         | 0.05     |


In [ ]:
# imports

import itertools
import os
import pandas as pd
import plotly.express as px
import random
import xarray as xr


In [ ]:
# input data variables

# data file
datapath = input('Enter the filepath of your input data file:')
# check data file exists
if not os.path.isfile(datapath):
    raise Exception(f'Unable to find the file "{datapath}"')

# output directory
outdir = input('Enter the filepath of the output directory where you want files saved:')
# check output directory exists
if not os.path.isdir(outdir):
    raise Exception(f'Unable to find the directory "{outdir}"')

# column names
mode0 = input('Enter the column name that corresponds to the first mode of your tensor (e.g. gene mode):')
mode1 = input('Enter the column name that corresponds to the second mode of your tensor (e.g. taxon mode):')
mode2 = input('Enter the column name that corresponds to the third mode of your tensor (e.g. sample mode):')
rep = input('Enter the column name that corresponds to replicate IDs:')
data = input('Enter the column name that corresponds to your normalized data:')

# read in csv
df = pd.read_csv(datapath)

# check column names match inputs
for column in [mode0, mode1, mode2, rep, data]:
    if column not in df.columns:
        raise Exception(f'Column name "{column}" not found in headers of file {datapath}')

# make column for combined sample-replicate ID
df['sample_replicate_id'] = df[mode2].astype(str) + '_' + df[rep].astype(str)

# tidy up dataframe
df = df[[mode0, mode1, 'sample_replicate_id', mode2, rep, data]].rename(columns={mode2: 'sample_id', rep: 'replicate_id', data: 'data'})

df


In [ ]:
# organize dataset into xarray.Dataset tensor

# create xr.Dataset
sample_df = df[['sample_replicate_id', 'sample_id', 'replicate_id']].drop_duplicates()
ds = xr.Dataset({
    'data': xr.DataArray.from_series(df.set_index([mode0, mode1, 'sample_replicate_id'])['data']), 
    'sample_id': xr.DataArray.from_series(sample_df.set_index('sample_replicate_id')['sample_id']), 
    'replicate_id': xr.DataArray.from_series(sample_df.set_index('sample_replicate_id')['replicate_id'])
})

# fill missing values with zeros
ds = ds.fillna(0)

# save to output directory
out_file = f'{outdir}/data-tensor.nc'
if not os.path.isfile(out_file):
    ds.to_netcdf(out_file)
    print(f'Tensor dataset ds saved as {out_file}')
else:
    choice = ''
    while choice not in ['y', 'n']:
        choice = input(f'WARNING: The file {out_file} already exists. Would you like to overwrite? (Y/N):').lower().strip()
    if choice == 'y':
        ds.to_netcdf(out_file)
        print(f'Tensor dataset ds saved as {out_file}')
    elif choice == 'n':
        print('Tensor dataset ds not saved.')

# show Dataset
ds


### Visualize data tensor (optional)

Use plotly to visualize your data tensor. 

NOTE: This visualization tool is not intended for tensors larger than a couple of hundred values per mode. If your tensor is larger than that, you must select a subset region you would like to visualize.

In [ ]:
# down-select subset of data tensor to visualize

viz_da = ds.data
display(dict(viz_da.sizes))

# choose dimensions
overall_max = 100
dims = {}
for mode in [mode0, mode1, 'sample_replicate_id']:
    maximum = min(viz_da[mode].size, overall_max)
    n = 0
    while n < 1 or n > maximum:
        n = int(input(f'Enter number of {mode} values to show (max = {maximum}):'))
    dims[mode] = n

# choose any data points you'd like to see
# (initialize idx here)

# down-select to only non-zero data
viz_df = viz_da.to_series().reset_index()
viz_df = viz_df[viz_df['data'] != 0]
for mode in [mode0, mode1, 'sample_replicate_id']:
    dims[mode] = min(dims[mode], viz_df[mode].nunique())

# look at the indices in each mode that have the most non-negative values across the other modes
candidate_idx = {mode0: [], mode1: [], 'sample_replicate_id': []}
for mode_pair in itertools.permutations([mode0, mode1, 'sample_replicate_id'], 2):
    count_df = viz_df.groupby(mode_pair[0])[mode_pair[1]].value_counts().sort_values(ascending=False).reset_index()
    for mode in mode_pair:
        candidate_idx[mode].append(set(count_df[mode].unique()[:dims[mode]]))

# pull out consensus indices
idx = {}
for mode in [mode0, mode1, 'sample_replicate_id']:
    idx[mode] = set()
    consensus = set.intersection(*candidate_idx[mode]).difference(idx[mode])
    alternates = set.union(*candidate_idx[mode]).difference(consensus.union(idx[mode]))
    while len(idx[mode]) < dims[mode]:
        idx[mode] = idx[mode].union(set(random.sample(sorted(consensus), min(len(consensus), dims[mode] - len(idx[mode])))))
        idx[mode] = idx[mode].union(set(random.sample(sorted(alternates), dims[mode] - len(idx[mode]))))
    idx[mode] = list(idx[mode])

# down-select data
viz_da = viz_da.sel({mode0: idx[mode0], mode1: idx[mode1], 'sample_replicate_id': idx['sample_replicate_id']})
viz_da

In [ ]:
# visualize selected data

# build 3d scatterplot
modes = [mode for mode, _ in sorted(dims.items(), key=lambda item:item[1])]
viz_df = viz_da.to_series().reset_index()
viz_df['abs_data'] = viz_df['data'].abs()
fig = px.scatter_3d(
    viz_df, x=modes[0], y=modes[2], z=modes[1], color='data', size='abs_data', opacity=0.5, 
    color_continuous_scale='RdBu_r', 
)
# update layout to improve readability
fig.update_layout(scene={
    'xaxis': {'showbackground': False, 'showticklabels': False}, 
    'yaxis': {'showbackground': False, 'showticklabels': False}, 
    'zaxis': {'showbackground': False, 'showticklabels': False}, 
    'aspectmode': 'data'}, 
    width=700, paper_bgcolor='#fff', plot_bgcolor='#fff'
)
fig.update_traces(marker={'line': {'color': None, 'width': 0}})
